# Iris dataset

In [ ]:
import numpy as np
from sklearn import datasets
import random
import matplotlib.pyplot as plt
import auxiliary_fun as a
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import time
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import random
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification as mk
from keras.metrics import Precision, Recall, AUC, MeanSquaredError
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation


## Load dataset

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X1 = X[y==0]
X2 = X[y==1]
y1 = y[y==0]
y2 = y[y==1]

Xor = np.concatenate((X1,X2))
yor = np.concatenate((y1,y2))
fig = pyplot.figure()
ax = Axes3D(fig)
ax.scatter(Xor[:,0], Xor[:,1], Xor[:,2], c = yor,cmap = "prism")
plt.savefig("iris_or.png",dpi=360)
pyplot.show()

### Obtaining the dominating dataset with $\varepsilon \le 0.5$

In [ ]:
start = time.time()
index = a.dominatingSet(Xor,yor,epsilon=0.5)
Xsub=Xor[index]
ysub=yor[index]
end = time.time()
print(end - start)
fig = pyplot.figure()
ax = Axes3D(fig)
ax.scatter(Xsub[:,0], Xsub[:,1], Xsub[:,2], c = ysub, cmap = "prism")
#plt.savefig("iris_dom.png",dpi=360)
pyplot.show()

### Random dataset

In [ ]:
#X=np.column_stack((Xor,yor))
#np.random.shuffle(X)
#Xor = X[:,0:4]
#yor = X[:,4]
#random_index = random.sample(range(len(Xor)),len(Xsub))
#Xrand = Xor[random_index]
#yrand = yor[random_index]
#iris_random=np.column_stack((Xrand,yrand))
#np.savetxt("iris_random2.txt",iris_random)
iris_random=np.loadtxt("data/iris_random.txt")
Xrand = iris_random[:,0:4]
yrand = iris_random[:,4]
fig = pyplot.figure()
ax = Axes3D(fig)
ax.scatter(Xrand[:,0], Xrand[:,1], Xrand[:,2], c = yrand, cmap = "prism")
#plt.savefig("iris_rand.png",dpi=360)
pyplot.show()

### Training a perceptron and evaluating

In [ ]:
epochs = 100
n_features=4

In [ ]:
l = []
for i in range(5):
    model1 = Sequential()
    model1.add(Dense(units=1, activation='sigmoid',input_shape=(n_features,)))
    model1.compile(optimizer="SGD", loss='MSE', metrics=['accuracy',Recall(), Precision(),AUC(),MeanSquaredError()])
    import time
    start = time.time()
    history1=model1.fit(Xor, yor,batch_size=1, 
                                #batch_size=1,
                                #validation_split = val_split, 
                                epochs=epochs,
                                verbose=False)
    end = time.time()
    print(abs(start-end))
    l.append(model1.evaluate(Xor,yor,verbose=0))
l = np.array(l)
print(np.min(l[:,1]))

In [ ]:
l2 = []
for i in range(5):
    model2 = Sequential()
    model2.add(Dense(units=1, activation='sigmoid',input_shape=(n_features,)))
    model2.compile(optimizer="SGD", loss='MSE', metrics=['accuracy',Recall(), Precision(),AUC(),MeanSquaredError()])
    import time
    start = time.time()
    history2=model2.fit(Xsub, ysub,batch_size=1, 
                                #batch_size=1,
                                #validation_split = val_split, 
                                epochs=200,
                                verbose=False)
    end = time.time()
    print(abs(start-end))
    l2.append(model2.evaluate(Xor,yor,verbose=0))
np.min(np.array(l2)[:,1])

In [ ]:
np.mean(np.array(l2)[:,5])

In [ ]:
l3 = []
for i in range(5):
    model3 = Sequential()
    model3.add(Dense(units=1, activation='sigmoid',input_shape=(n_features,)))
    model3.compile(optimizer="SGD", loss='MSE', metrics=['accuracy',Recall(), Precision(),AUC(),MeanSquaredError()])
    history3=model3.fit(Xrand, yrand, 
                        batch_size=1, 
                        epochs=epochs, 
                        verbose=False)
    l3.append(model3.evaluate(Xor,yor,verbose=0))
print(np.min(np.array(l3)[:,1]))

### Hausdorff distance and the bottleneck distance

In [ ]:
from scipy.spatial.distance import directed_hausdorff
print("Hausdorff distance between the original dataset and the dominating dataset: ",max(directed_hausdorff(Xor, Xsub)[0], directed_hausdorff(Xsub, Xor)[0]))
print("Hausdorff distance between the original dataset and the random dataset: ",max(directed_hausdorff(Xor, Xrand)[0], directed_hausdorff(Xrand, Xor)[0]))

In [ ]:
from ripser import ripser
diagrams_or = ripser(Xor,maxdim=2)['dgms']
diagrams_Sub = ripser(Xsub,maxdim=2)['dgms']
diagrams_Rand = ripser(Xrand,maxdim=2)['dgms']

In [ ]:
import gudhi as g
message = "Bottleneck distance for dominating dataset and dimension 0 =" + '%.2f' % g.bottleneck_distance(diagrams_or[0], diagrams_Sub[0])
print(message)
message = "Bottleneck distance for dominating dataset and dimension 1 =" + '%.2f' % g.bottleneck_distance(diagrams_or[1], diagrams_Sub[1])
print(message)
message = "Bottleneck distance for Random dataset and dimension 0 =" + '%.2f' % g.bottleneck_distance(diagrams_or[0], diagrams_Rand[0])
print(message)
message = "Bottleneck distance for Random dataset and dimension 1 =" + '%.2f' % g.bottleneck_distance(diagrams_or[1], diagrams_Rand[1])
print(message)